In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
import itertools
import re
from collections import Counter

In [36]:
with open("/home/peter/Downloads/AdventOfCode/day22Input.txt") as fh:
    lines = fh.readlines()
#with open("/home/peter/Downloads/AdventOfCode/day22Test.txt") as fh:
#    lines = fh.readlines()
#with open("/home/peter/Downloads/AdventOfCode/day22TestT2.txt") as fh:
#    lines = fh.readlines()

In [37]:
ons = []
x1s = []
x2s = []
y1s = []
y2s = []
z1s = []
z2s = []
for line in lines:
    m =  re.match("(on|off) x=([-0-9]*)\.\.([-0-9]*),y=([-0-9]*)\.\.([-0-9]*),z=([-0-9]*)\.\.([-0-9]*)",line)
    if m:
        #print("matched")
        on,x1,x2,y1,y2,z1,z2 = m.groups()
        #print(m.groups())
        on = 1 if on == "on" else 0
        ons.append(on)
        x1s.append(int(x1))
        x2s.append(int(x2))
        y1s.append(int(y1))
        y2s.append(int(y2))
        z1s.append(int(z1))
        z2s.append(int(z2))
    else:
        print("error on line ",line)
print(len(lines))
print(len(ons))
print(x1s)

420
420
[-27, 1, -14, -15, -13, -8, -24, -47, -45, 1, -36, -17, 32, -7, 34, -7, -39, -4, -19, -18, -38704, -11481, -14104, 18108, -5122, -82960, 41884, -92027, -82663, -56376, -29571, -71506, -18850, 30387, 42170, -50232, -29117, 42489, -14477, -73250, -62732, -49882, -70725, -90112, 57934, 13322, 8479, 67524, 59622, 32146, 55538, -31760, 52583, 41338, -58140, 32787, -32867, -80173, -15315, -5631, -21782, 65929, -67840, -52847, 26874, 59596, -51198, -77282, 34838, 867, -80362, -33491, -85897, -39592, 14593, -71776, -29480, -9536, 41264, -27995, -3353, -67845, 11458, -68163, 34234, -36022, 53817, -6929, -34122, -27336, 32912, -23447, -10239, -75920, 67446, -5274, -12309, 1595, 61329, 37007, -77958, 46737, -80607, 66460, -2275, 39928, 39864, 12849, -87576, 63009, -64161, -85354, -43821, 42353, -79497, 35788, 65259, -22398, -55447, 30258, -79399, 32196, 41920, -10445, -1049, -24084, 15238, 20491, 34614, 458, -10794, 25505, -7778, -36415, -46832, 3532, 23374, -61323, 32417, -23418, -23108,

In [38]:
msize = 110
stage = np.zeros(shape=(msize,msize,msize),dtype=np.int64)

In [39]:
limit = (msize-8)/2
offset = limit+5
for i in range(len(ons)):
    # ignore if less than
    if x2s[i] < -limit or y2s[i] < -limit or z2s[i] < -limit:
        continue
    elif x1s[i] > limit or y1s[i] > limit or z1s[i] > limit:
        continue
    x1 = int(max(0,x1s[i]+offset))
    y1 = int(max(0,y1s[i]+offset))
    z1 = int(max(0,z1s[i]+offset))
    x2 = int(min(msize,x2s[i]+offset+1)) # +1 because clopen
    y2 = int(min(msize,y2s[i]+offset+1))
    z2 = int(min(msize,z2s[i]+offset+1))
    print("setting ",x1,x2,y1,y2,z1,z2,ons[i])
    stage[x1:x2,y1:y2,z1:z2] = ons[i]

setting  29 84 15 61 47 95 1
setting  57 105 35 89 53 103 1
setting  42 90 40 88 35 90 1
setting  41 92 44 94 21 66 1
setting  43 96 44 95 36 85 1
setting  48 96 49 101 56 106 1
setting  32 78 56 103 24 77 1
setting  9 59 59 106 33 86 1
setting  11 57 37 88 8 62 1
setting  57 106 50 102 28 78 1
setting  20 33 31 48 93 103 0
setting  39 90 23 76 36 83 1
setting  88 105 10 30 75 88 0
setting  49 103 54 106 47 97 1
setting  90 106 72 87 82 95 0
setting  49 99 23 77 25 80 1
setting  17 33 62 81 13 27 0
setting  52 101 45 90 34 85 1
setting  37 53 67 81 52 72 0
setting  38 92 57 104 46 99 1


In [40]:
np.count_nonzero(stage)

587785

In [41]:
cubes = Counter()
for i in range(len(ons)):
    sign = ons[i]
    sign = 1 if sign == 1 else -1
    x1 = x1s[i]
    y1 = y1s[i]
    z1 = z1s[i]
    x2 = x2s[i]
    y2 = y2s[i]
    z2 = z2s[i]
    newCubes = Counter()
    for (cx1,cx2,cy1,cy2,cz1,cz2),count in cubes.items():
        #print('.')
        px1 = max(x1,cx1)
        px2 = min(x2,cx2)
        py1 = max(y1,cy1)
        py2 = min(y2,cy2)
        pz1 = max(z1,cz1)
        pz2 = min(z2,cz2)
        if px1 <= px2 and py1 <= py2 and pz1 <= pz2:
            #print("setting ",px1,px2,py1,py2,pz1,pz2,count)
            newCubes[(px1,px2,py1,py2,pz1,pz2)] -= count
    if sign > 0:
        #print("foo")
        newCubes[(x1,x2,y1,y2,z1,z2)] += sign
    cubes.update(newCubes)



In [42]:
dsum = 0
for (cx1,cx2,cy1,cy2,cz1,cz2),count in cubes.items():
    dsum += (cx2-cx1+1)*(cy2-cy1+1)*(cz2-cz1+1)*count
print(dsum)
#print(cubes.items())

1167985679908143


In [43]:
len("2758514936282235")

16

In [44]:
cubes = Counter()
for line in lines:
    nsgn = 1 if line.split()[0] == "on" else -1
    nx0, nx1, ny0, ny1, nz0, nz1 = map(int, re.findall("-?\d+", line))

    update = Counter()
    for (ex0, ex1, ey0, ey1, ez0, ez1), esgn in cubes.items():
        ix0 = max(nx0, ex0); ix1 = min(nx1, ex1)
        iy0 = max(ny0, ey0); iy1 = min(ny1, ey1)
        iz0 = max(nz0, ez0); iz1 = min(nz1, ez1)
        if ix0 <= ix1 and iy0 <= iy1 and iz0 <= iz1:
            update[(ix0, ix1, iy0, iy1, iz0, iz1)] -= esgn
    if nsgn > 0:
        update[(nx0, nx1, ny0, ny1, nz0, nz1)] += nsgn
    cubes.update(update)

print(sum((x1 - x0 + 1) * (y1 - y0 + 1) * (z1 - z0 + 1) * sgn
          for (x0, x1, y0, y1, z0, z1), sgn in cubes.items()))

1167985679908143
